In [ ]:
import pandas as pd
import json
import pytz
import nltk
import numpy as np
import string
from nltk.tokenize import word_tokenize
import re
nltk.download('stopwords')

# Import Chat Data

In [ ]:
#import chatdata from Apex here
chatdata = pd.read_csv("2025-08-01_kato_messages.csv")

In [ ]:
chatdata

In [ ]:
#sort by CONVERSATION_ID and TIME to maintain order
chatdata_sorted = chatdata.sort_values(by=["CONVERSATION_ID", "TIME"])

In [ ]:
chatdata_sorted["TIME"] = pd.to_datetime(chatdata_sorted["TIME"], format="%d-%b-%y %I.%M.%S.%f %p")

In [ ]:
chatdata_sorted["USER"] = chatdata_sorted["MESSAGE_ID"].str.split("_").str[0]
chatdata_sorted

In [ ]:
chatdata_sorted[chatdata_sorted["USER"] == "ffd106a9-3ae4-419d-b0d4-85ebb75e0df2"]

In [ ]:
def clean_json_string(json_str):
    #remove any unexpected characters around JSON fields
    json_str = re.sub(r'(\w+)\s*"', r'\1"', json_str)
    return json_str

def safe_json_loads(json_str):
    try:
        # Clean the JSON string first
        cleaned_str = clean_json_string(json_str)
        return json.loads(cleaned_str)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Problematic JSON string: {json_str[:500]}...")
        return None

#read in the raw LRS data here
df = pd.read_csv('kato_raw_July.csv')

#parse the JSON column
df['parsed_json'] = df['Payload'].apply(safe_json_loads)

#remove any rows where parsing failed
df = df.dropna(subset=['parsed_json'])

In [ ]:
#Checks that process did not remove any rows from original csv
len(df['parsed_json']), len(df['Payload'])

# Anonymize the Data

In [ ]:
# Create a set for unique email-name combinations
unique_combinations = set()

for json_obj in df['parsed_json']:
   if json_obj and 'actor' in json_obj:
       email = json_obj['actor'].get('mbox')
       name = json_obj['actor'].get('name')
       if email and name:
           combo = (email, name)
           unique_combinations.add(combo)

In [ ]:
# Create a dictionary to map emails to anonymous identifiers
email_mapping = {}
counter = 1

def anonymize_by_email(json_obj):
   global counter
   
   if json_obj and 'actor' in json_obj:
       original_email = json_obj['actor'].get('mbox')
       
       # If this email hasn't been mapped yet, create a new mapping
       if original_email and original_email not in email_mapping:
           email_mapping[original_email] = f"Person {counter}"
           counter += 1
       
       # Replace the email with the anonymous identifier
       if original_email:
           json_obj['actor']['mbox'] = email_mapping[original_email]
           
   return json_obj

# Apply the anonymization
df['parsed_json'] = df['parsed_json'].apply(anonymize_by_email)

# Print the mapping (optional, to keep track of which email maps to which person)
print("Email to Person mapping:")
for email, person in email_mapping.items():
   print(f"{email} -> {person}")

# Print total number of unique people
print(f"\nTotal number of unique people: {len(email_mapping)}")

In [ ]:
# Create a dictionary to maintain consistent name mapping
name_mapping = {}
counter = 1

def anonymize_name(json_obj):
    global counter
    
    if json_obj and 'actor' in json_obj:
        original_name = json_obj['actor'].get('name')
        
        # If this name hasn't been mapped yet, create a new mapping
        if original_name and original_name not in name_mapping:
            name_mapping[original_name] = f"Person {counter}"
            counter += 1
        
        # Replace the name with the anonymous version
        if original_name:
            json_obj['actor']['name'] = name_mapping[original_name]
            
        # Remove email
        if 'mbox' in json_obj['actor']:
            del json_obj['actor']['mbox']
            
    return json_obj

# Apply the anonymization
df['parsed_json'] = df['parsed_json'].apply(anonymize_name)

# Print the mapping (optional, to keep track of who is who)
print("Name mapping:", name_mapping)

In [ ]:
# Convert the name mapping dictionary to a DataFrame
mapping_df = pd.DataFrame.from_dict(name_mapping, orient='index', columns=['Anonymous_ID'])
mapping_df.index.name = 'Original_Name'

# Save to CSV
mapping_df.to_csv('name_mapping_July25_Kato.csv')

# Print to verify
print("Name mapping saved to 'name_mapping.csv'")
print("\nPreview of the mapping DataFrame:")
print(mapping_df.head())

In [ ]:
# Extract registrations
registrations = df['parsed_json'].apply(lambda x: 
    x.get('context', {}).get('registration') if x else None
)

# Print the registrations
print("Registrations:", registrations)

In [ ]:
df["parsed_json"].to_csv('anonymized_data_July25_Kato.csv', index=False)

## Editing LRS Data and Parsing JSON

In [ ]:
normalized_df = pd.read_csv("anonymized_data_July25_Kato.csv")

In [ ]:
#clean JSON and parse data
def clean_and_parse_json(x):
    if pd.isnull(x):
        return None

    try:
        #replace PHP symbol with single quote
        x = x.replace("â€™", "'")

        #replace "True" with "true"
        x = x.replace("True", "true").replace("False", "false").replace("None", "null")

        #replace single quotes with double quotes (this is going to throw errors for stuff like "Mr. Kato's" with the single quote. Go into Excel and Replace "Mr. Kato's" -> "Mr. Katos" along with other words too
        #reload the csv, then run this window again until all errors below are gone. Read carefully, as it will tell you where the error is.. e.g. "Mr. Kato"s"
        #there might be an easier work around for this, as it is quite tedious
        x = x.replace("'", '"')

        #white space removal
        x = x.strip()

        #parse cleaned JSON
        return json.loads(x)
        
    except json.JSONDecodeError as e:
        #log rows where it isn't parsing correctly
        print(f"JSONDecodeError: {e} for row: {x}")
        return None

#apply cleaning and parsing function to dataset column
normalized_df["parsed_json"] = normalized_df["parsed_json"].apply(clean_and_parse_json)

#create column indicating whether the row was parsed correctly (good for debugging)
normalized_df["is_valid_json"] = normalized_df["parsed_json"].apply(lambda x: isinstance(x, dict))

#normalize data into columns based on features
normalized_df = pd.json_normalize(normalized_df["parsed_json"].where(normalized_df["is_valid_json"], None))

In [ ]:
#convert timestamp to datetime if it isn't already
normalized_df["timestamp"] = pd.to_datetime(normalized_df["timestamp"])

#check if timestamp (does not have a timezone)
if normalized_df["timestamp"].dt.tz is None:
    #localize to UTC first
    normalized_df["timestamp"] = normalized_df["timestamp"].dt.tz_localize("UTC")

#convert to GMT-8 (Etc/GMT+8 corresponds to GMT-8)
gmt_8 = pytz.timezone("Etc/GMT+8")
normalized_df["timestamp_gmt8"] = normalized_df["timestamp"].dt.tz_convert(gmt_8)

#ensure timestamp_gmt8 remains timezone-aware (remove the unnecessary pd.to_datetime())
normalized_df = normalized_df[normalized_df["timestamp_gmt8"].notna()]


In [ ]:
#manually setting the slides to keys with chronological order
slide_mapping = {
    "Virtual Patient Case: Mr. Kato": 1,
    "Introduction": 2,
    "Learning Objectives": 3,
    "Case Overview": 4,
    "Reflecting Back": 5,
    "Ruling Out a Diagnosis": 6,
    "1.6 Ruling Out a Diagnosis - Life Threatening Conditions": 7,
    "1.6 Ruling Out a Diagnosis - Sight Threatening Conditions": 8,
    "Meeting the Patient": 9,
    "Confidence Check I: Engage With Mr. Kato": 10,
    "Practice: Conversational History Taking": 11,
    "Feedback Survey": 12,
    "Choose a Path: Hx or DDx": 13,
    "History Taking": 14,
    "Mr. Katos History": 15,
    "Mr. Katos Ocular History": 16,
    "Practice: Refractive Error Classification": 17,
    "Refractive Error Classification": 18,
    "Medical and Social History": 19,
    "Identifying Vision Loss": 20,
    "Initial Differential Diagnosis": 21,
    "Initial DDX: Vascular Occlusion": 22,
    "Initial DDX: Diabetic Retinopathy": 23,
    "Initial DDX: Cornea Ulcer": 24,
    "Initial DDX: Optic Neuritis": 25,
    "Initial DDX: GCA": 26,
    "Initial DDX: Cataract": 27,
    "Initial DDX: AMD": 28,
    "Initial DDX: Corneal Scarring": 29,
    "Initial DDX: Retinal Detachment": 30,
    "Eye Examination": 31,
    "1.20 Eye Examintation - Visual Acuity": 32,
    "1.20 Eye Examintation - Pupils": 33,
    "1.20 Eye Examintation - Confrontational Visual Fields": 34,
    "1.20 Eye Examintation - Extraocular Movements": 35,
    "1.20 Eye Examintation - Slit Lamp": 36,
    "1.20 Eye Examintation - Direct Ophthalmoscope": 37,
    "Mr. Katos Eye Exam": 38,
    "1.21 Mr Katos Eye Exam - Slit Lamp of Mild Cataract": 39,
    "1.21 Mr Katos Eye Exam - Terminology": 40,
    "1.21 Mr Katos Eye Exam - Snellen Chart": 41,
    "Direct Ophthalmoscopy": 42,
    "Direct Ophthalmoscopy: In Practice": 43,
    "Dilated Fundus Anatomy": 44,
    "Review: Dilated Fundus Anatomy": 45,
    "Mr. Kato’s Dilated Fundus Exam": 46,
    "Confidence Check II: Differential Diagnosis vs. Chatting with Mr. Kato": 47,
    "Differential Diagnosis": 48,
    "Patient Communication": 49,
    "Patient Referral": 50,
    "Practice: Matching Treatments": 51,
    "Epilogue": 52,
    "Learning Review": 53,
}

#creating slide number column, mapped to appropriate title
normalized_df["slide_number"] = normalized_df["object.definition.name.und"].map(slide_mapping)

In [ ]:
#text preprocessing function
def preprocess_text(text):
    stop_words = set(stopwords.words("english"))

    #if response is one character or punctuation, then classify it as "one_character"
    if len(text.strip()) == 1 or text.strip() in string.punctuation:
        return ["one_character"]

    #convert text to lowercase, remove punctuation
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

#create new column as "None"
normalized_df.loc[:, "processed_response"] = None

#ensure the verb is "answered" and the description is either "essay" or "text entry interaction" when preprocessing
is_essay_or_text = normalized_df.loc[(normalized_df["object.definition.description.und"] == "Essay") | (normalized_df["object.definition.description.und"] == "Text Entry Interaction")]

#apply preprocessing to the relevant rows only
normalized_df.loc[is_essay_or_text.index, "processed_response"] = is_essay_or_text["result.response"].apply(preprocess_text)

#ensure all other columns remain unchanged in normalized_df
normalized_df

## Removing Non Students

In [ ]:
#check the name mapping file we made above to see which names are team members, then remove them here.

normalized_df = normalized_df[~normalized_df["actor.name"].isin(["Person 10", "Person 14", "Person 21", "Person 22"])]

In [ ]:
normalized_df

## Combining datasets

In [ ]:
#warning will be thrown at the bottom, this is okay.

#ensure TIME column is correctly formatted without changing timezone
chatdata_sorted["TIME"] = pd.to_datetime(chatdata_sorted["TIME"], format="%d-%b-%y %I.%M.%S.%f %p")

#rename column to match normalized_df timestamp_gmt8
chatdata_sorted = chatdata_sorted.rename(columns={"TIME": "timestamp_gmt8"})

#ensure both timestamp_gmt8 columns are tz-naive (removes timezone for sorting compatibility)
normalized_df["timestamp_gmt8"] = normalized_df["timestamp_gmt8"].dt.tz_localize(None)
chatdata_sorted["timestamp_gmt8"] = chatdata_sorted["timestamp_gmt8"].dt.tz_localize(None)

#merge chatdata_sorted with normalized_df using context.registration and USER
merged_chat = chatdata_sorted.merge(
    normalized_df[['actor.name', 'context.registration']], 
    left_on="USER", 
    right_on="context.registration", 
    how="left"
)

#combine both datasets with specific columns
final_combined_df = pd.concat([
    normalized_df[['actor.name', 'context.registration', 'timestamp_gmt8', 'object.definition.name.und', 'verb.display.en-US', 'result.duration', 'slide_number']],  # Original actor data
    merged_chat[['actor.name', 'context.registration', 'timestamp_gmt8', 'ROLE', 'CONTENT', 'USER']]  # Merged chat data
], ignore_index=True)

#ensure timestamps are tz-naive before sorting
final_combined_df["timestamp_gmt8"] = final_combined_df["timestamp_gmt8"].dt.tz_localize(None)

#sort everything by actor.name and timestamp_gmt8 to correctly slot in chat data
final_combined_df = final_combined_df.sort_values(by=["actor.name", "timestamp_gmt8"])

# **Remove duplicate timestamps** by keeping only the first instance per actor & timestamp
final_combined_df = final_combined_df.drop_duplicates(subset=["actor.name", "timestamp_gmt8"], keep="first")

In [ ]:
final_combined_df[final_combined_df["actor.name"] == "Person 3"]

In [ ]:
final_combined_df = final_combined_df[final_combined_df['actor.name'].notna() & (final_combined_df['actor.name'] != '')]

In [ ]:
final_combined_df

#when wanting to view date in a proper format in excel
final_combined_df["timestamp_gmt8"] = "'" + final_combined_df["timestamp_gmt8"].astype(str)

final_combined_df.to_csv("StudentDataFeb11-21.csv", index=False)

In [ ]:
final_combined_df.to_csv("ShinyFinalDataJuly25.csv", index=False)